# VTA 测试

In [1]:
import logging
from mxnet.gluon.model_zoo.vision import get_model

from tvm import relay
from tvm.driver import tvmc
from tvm.driver.tvmc import TVMCModel, TVMCPackage, TVMCResult

In [2]:
pretrained = True
shape_dict = {'data': (1, 3, 224, 224)}
model_name = 'mobilenet1.0'
out_dir = 'outputs'
logging.basicConfig(filename=f'{out_dir}/{model_name}.log')
logger = logging.getLogger(name='logger')
logger.setLevel(logging.DEBUG)

model = get_model(model_name, pretrained=pretrained)
mod, params = relay.frontend.from_mxnet(model, shape_dict)
model = TVMCModel(mod, params)
tvmc.compile(model, target="llvm", package_path="whatever")
new_package = TVMCPackage(package_path="whatever")
result = tvmc.run(new_package, device='cpu') #Step 3: Run
logger.info(model.mod['main'].astext()) # 记录 mod
print(result.format_times())
result.save(f'{out_dir}/{model_name}_resluts')
model.save(f"{out_dir}/{model_name}.params")

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  41.2985      42.7237      51.0308      22.5848       8.2324   
               


In [4]:
tuning_records = tvmc.tune(model, target="llvm")

[Task  1/21]  Current/Best:    0.00/   0.00 GFLOPS | Progress: (0/476) | 0.00 s

/media/pc/data/4tb/lxw/anaconda3/envs/tvm-mxnet/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[Task  1/21]  Current/Best:   50.58/ 166.81 GFLOPS | Progress: (48/476) | 16.00 s

/media/pc/data/4tb/lxw/anaconda3/envs/tvm-mxnet/lib/python3.10/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


[Task  1/21]  Current/Best:   21.20/ 253.41 GFLOPS | Progress: (264/476) | 91.10 s Done.
[Task  2/21]  Current/Best:   43.16/ 308.59 GFLOPS | Progress: (476/476) | 179.28 s Done.
[Task  3/21]  Current/Best:   15.24/ 328.74 GFLOPS | Progress: (476/476) | 171.79 s Done.
[Task  5/21]  Current/Best:   73.62/ 354.71 GFLOPS | Progress: (48/476) | 15.28 s s Done.
[Task  5/21]  Current/Best:   24.46/ 354.71 GFLOPS | Progress: (476/476) | 215.24 s Done.
[Task  7/21]  Current/Best:   33.14/  96.04 GFLOPS | Progress: (48/476) | 22.35 s s Done.
[Task  7/21]  Current/Best:   33.52/ 235.03 GFLOPS | Progress: (476/476) | 234.01 s Done.
[Task  9/21]  Current/Best:   25.25/ 153.50 GFLOPS | Progress: (48/476) | 25.32 s s Done.
[Task 10/21]  Current/Best:    1.60/  79.94 GFLOPS | Progress: (48/476) | 23.82 s s Done.
[Task 11/21]  Current/Best:    0.00/   0.00 GFLOPS | Progress: (0/476) | 0.00 s96 s Done.
[Task 11/21]  Current/Best:   54.27/ 136.25 GFLOPS | Progress: (476/476) | 256.76 s Done.
[Task 13/21

In [5]:
tvmc_package = tvmc.compile(model, target="llvm", tuning_records=tuning_records)
result = tvmc.run(tvmc_package, device="cpu")
print(result)

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  22.4732      20.9165      54.9719       9.5562      13.0234   
               
Output Names:
 ['output_0']
